In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [ ]:
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
print(stopwords.words('english'))
l=stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
data_train=pd.read_csv('/content/twitter_training.csv')
data_test=pd.read_csv('/content/twitter_validation.csv')

In [ ]:
data_train.head()
data_train.shape

(74681, 4)

In [ ]:
col=['Tweet ID',
'entity',
'target',
'Tweet']
data_train.columns =col
data_train.head()

,Tweet ID,entity,target,Tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
df=pd.DataFrame(data_train)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tweet ID  74681 non-null  int64 
 1   entity    74681 non-null  object
 2   target    74681 non-null  object
 3   Tweet     73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
df=df[df['Tweet'].str.len()!=0]


In [ ]:
df['Tweet'] = df['Tweet'].astype(str)
df['Tweet'] = df['Tweet'].str.lower()
print(df['Tweet'])
df.head()

0        i am coming to the borders and i will kill you...
1        im getting on borderlands and i will kill you ...
2        im coming on borderlands and i will murder you...
3        im getting on borderlands 2 and i will murder ...
4        im getting into borderlands and i can murder y...
                               ...                        
74676    just realized that the windows partition of my...
74677    just realized that my mac window partition is ...
74678    just realized the windows partition of my mac ...
74679    just realized between the windows partition of...
74680    just like the windows partition of my mac is l...
Name: Tweet, Length: 74681, dtype: object


,Tweet ID,entity,target,Tweet
0,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
nlp=spacy.load("en_core_web_sm")
def preprocess(text):
    doc=nlp(text)
    new_text=[]
    for word in doc:
        if word.is_stop or word.is_punct or word.text=="im":
            continue
        new_text.append(word.lemma_)
    return " ".join(new_text)

In [ ]:
df['target'].value_counts()

target
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

In [ ]:
df['stemmed_content']=df['Tweet'].apply(preprocess)
df.head()


,Tweet ID,entity,target,Tweet,stemmed_content
0,2401,Borderlands,Positive,i am coming to the borders and i will kill you...,come border kill
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,m get borderland kill
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,m come borderland murder
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...,m get borderland murder


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['target']=encoder.fit_transform(df['target'])
df.head()

,Tweet ID,entity,target,Tweet,stemmed_content
0,2401,Borderlands,3,i am coming to the borders and i will kill you...,come border kill
1,2401,Borderlands,3,im getting on borderlands and i will kill you ...,m get borderland kill
2,2401,Borderlands,3,im coming on borderlands and i will murder you...,m come borderland murder
3,2401,Borderlands,3,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder
4,2401,Borderlands,3,im getting into borderlands and i can murder y...,m get borderland murder


In [ ]:
df['target'].value_counts()
# 0-->Irrelevant
# 1--> Negative
# 2-->Positive
# 3-->Neutral

target
1    22542
3    20831
2    18318
0    12990
Name: count, dtype: int64

In [ ]:
X=df['stemmed_content']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape,y.shape,y_train.shape,y_test.shape)

(74681,) (52276,) (22405,) (74681,) (52276,) (22405,)


In [ ]:
vectorizer=TfidfVectorizer()

In [ ]:
# Fit and transform the training data
X_train = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test = vectorizer.transform(X_test)


In [ ]:
print(X_train)

  (0, 17739)	0.2460513065432312
  (0, 9838)	0.2460513065432312
  (0, 13474)	0.20842146822772653
  (0, 12275)	0.23880538324650571
  (0, 21124)	0.18784871149964488
  (0, 19310)	0.12194856666751817
  (0, 6446)	0.16893241411145973
  (0, 3684)	0.18072746918797308
  (0, 16467)	0.2629834369600888
  (0, 17306)	0.1761337479722038
  (0, 3097)	0.11013735913245903
  (0, 19027)	0.21023177040398586
  (0, 5553)	0.21023177040398586
  (0, 16254)	0.2178937660691438
  (0, 19789)	0.22209170706069778
  (0, 6866)	0.20161308494076105
  (0, 273)	0.24228572906144175
  (0, 22826)	0.1514859931077614
  (0, 4443)	0.17110521445707502
  (0, 20499)	0.24228572906144175
  (0, 19558)	0.13654096588443382
  (0, 17471)	0.18603840932338558
  (0, 4030)	0.16963542379640112
  (0, 13727)	0.23614664519204945
  (0, 5854)	0.14665352551672525
  :	:
  (52272, 16931)	0.16431203356021346
  (52273, 20946)	0.46348653821333946
  (52273, 7528)	0.47941088898324136
  (52273, 20437)	0.44975924229837955
  (52273, 9198)	0.2944040116583137
  (5

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Create a Decision Tree classifier object
model = DecisionTreeClassifier()

In [ ]:
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
X_test_predict=model.predict(X_test)
X_test_acc=accuracy_score(y_test,X_test_predict)
print(X_test_acc)

0.7839767908948896


In [ ]:
X_test_predict

array([0, 3, 3, ..., 3, 0, 2])